In [1]:
library(Seurat)
library(Matrix)
library(reticulate)
library(glmpca)
library(peakRAM)
library(scry)
library(SingleCellExperiment)
library(glmGamPoi)
library(harmony)

Warning message in fun(libname, pkgname):
“rgeos: versions of GEOS runtime 3.9.1-CAPI-1.14.2
and GEOS at installation 3.9.1dev-CAPI-1.14.1differ”
Attaching SeuratObject

Attaching sp

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods

# Load data

In [3]:
mtx <- readMM('../datasets/pbmc_ssc/mat.mtx')
meta <- read.csv('../datasets/pbmc_ssc/meta.csv', row.names=1)
colnames(mtx) <- rownames(meta)
rownames(mtx) <- 1:nrow(mtx)
covmat <- model.matrix(as.formula('~factor(Method)'), meta)
covmat <- covmat[,2:ncol(covmat)]
meta[,paste0('batch_',1:ncol(covmat))] <- covmat

# lognorm - nobatch

In [7]:
benchmark_seurat_log <- function(obj){
    obj <- NormalizeData(obj)
    obj <- FindVariableFeatures(obj, verbose=FALSE, nfeatures=2000)
    obj <- ScaleData(obj, verbose=FALSE)
    obj <- RunPCA(obj, verbose=FALSE)
    write.csv(Embeddings(obj), 'pca_coord/pbmcssc/pca_log_nobatch.csv')
    obj
}

In [8]:
obj <- CreateSeuratObject(mtx, meta.data=meta)
result <- peakRAM(x <- benchmark_seurat_log(obj))

In [9]:
result[,c('Elapsed_Time_sec', 'Peak_RAM_Used_MiB')]

,Elapsed_Time_sec,Peak_RAM_Used_MiB
,<dbl>,<dbl>
1,10.804,732.5


# glmpca - nobatch

In [4]:
benchmark_scry_glmpca <- function(obj){
    obj <- devianceFeatureSelection(obj, assay="counts", sorted=TRUE)
    obj2 <- obj[1:2000,]
    obj2 <- GLMPCA(obj2, 50, assay="counts", minibatch="stochastic")
    write.csv(metadata(obj2)$glmpca$factors, 'pca_coord/pbmcssc/pca_glmpca_nobatch.csv')
}

In [5]:
obj <- SingleCellExperiment(list(counts=mtx))
colData(obj)[,paste0('batch_',1:ncol(covmat))] <- covmat
result <- peakRAM(x <- benchmark_scry_glmpca(obj))

In [6]:
result[,c('Elapsed_Time_sec', 'Peak_RAM_Used_MiB')]

,Elapsed_Time_sec,Peak_RAM_Used_MiB
,<dbl>,<dbl>
1,1114.301,1485.8
